# Groq API 써보기

Groq의 빠른 추론 엔진을 LangChain에서 체험하는 코드를 소개합니다.    
langchain_groq를 통해 쉽게 연결할 수 있습니다.

In [1]:
!pip install langchain langchain_groq langchain_community pymupdf pypdf beautifulsoup4 requests

     ---------------------------------------- 108.9/108.9 kB ? eta 0:00:00



[notice] A new release of pip available: 22.3 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


현재 사용 가능한 주요 모델과 API 제한은 아래와 같습니다.


| ID                                     | Requests per Minute | Requests per Day | Tokens per Minute | Tokens per Day |
|----------------------------------------|---------------------|------------------|-------------------|----------------|
| gemma-7b-it                            | 30                  | 14,400           | 15,000            | 500,000        |
| gemma2-9b-it                           | 30                  | 14,400           | 15,000            | 500,000        |
| llama-3.1-70b-versatile                | 30                  | 14,400           | 18,000            | 500,000        |
| llama-3.1-8b-instant                   | 30                  | 14,400           | 20,000            | 500,000        |
| llama-3.2-11b-text-preview             | 30                  | 7,000            | 7,000             | 500,000        |
| llama-3.2-11b-vision-preview           | 30                  | 7,000            | 7,000             | 500,000        |
| llama-3.2-1b-preview                   | 30                  | 7,000            | 7,000             | 500,000        |
| llama-3.2-3b-preview                   | 30                  | 7,000            | 7,000             | 500,000        |
| llama-3.2-90b-text-preview             | 30                  | 7,000            | 7,000             | 500,000        |
| llama-3.2-90b-vision-preview           | 15                  | 3,500            | 7,000             | 250,000        |
| llava-v1.5-7b-4096-preview             | 30                  | 14,400           | 30,000            | (No limit)     |
| mixtral-8x7b-32768                     | 30                  | 14,400           | 5,000             | 500,000        |


In [2]:
from langchain_groq import ChatGroq # Groq-LangChain 연결

from langchain.document_loaders import WebBaseLoader
# from langchain_community.document_loaders import WebBaseLoader

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

import time

USER_AGENT environment variable not set, consider setting it to identify your requests.


간단한 체인을 만들고 실행합니다.

https://console.groq.com/keys 에서 키를 생성해야 합니다.

In [4]:
import os
os.environ['GROQ_API_KEY'] = ''


chat = ChatGroq(
    temperature=0.1,
    model="llama-3.1-70b-versatile",
)

url = "https://wow.groq.com/why-groq/"
loader = WebBaseLoader(url)

docs = loader.load()
question = "Groq 엔진의 LLM 추론 속도가 빠른 이유는 무엇인가요? 한국어로 답변하세요."


system = "Answer the question from given contexts. Answer in Korean."
human = """
Context: {context}

---

Question: {question}
"""

prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | chat | StrOutputParser()


start = time.time()
result = chain.invoke(
          {"context":docs[0].page_content,
            'question':question})
print(result)

end = time.time()
elapsed_time = end - start
print(f"Elapsed Time: {elapsed_time} seconds")

Groq의 LPU (Language Processing Unit) 기술 덕분에 빠른 추론 속도를 제공합니다. LPU는 기존의 그래픽 처리를 위한 GPU와 달리, AI 추론과 언어 처리를 위해 특별히 설계되었습니다. 이로 인해 LPU는 기존의 GPU보다 더 빠른 추론 속도와 더 나은 에너지 효율성을 제공할 수 있습니다.
Elapsed Time: 0.8567409515380859 seconds


## Summarization

요약은 LLM의 아주 중요한 기능 중 하나입니다.   
일반적으로, LLM의 Abstractive Summarization은 3개의 방법을 사용합니다.

- Stuff : 전체 코퍼스를 하나의 프롬프트에 넣고 요약 생성하기
- Map-Reduce : 코퍼스를 청크로 분리하고, 각 청크의 요약을 생성한 뒤 합치기
- Refine: 코퍼스를 청크로 분리하고, 순차적으로 읽으며 요약 업데이트하기



## Stuff

gemma 2 모델을 이용해 요약을 수행해 보겠습니다.

In [7]:
# Stuff
example_URL='https://ai.meta.com/blog/llama-3-2-connect-2024-vision-edge-mobile-devices/'

loader = WebBaseLoader(example_URL)
docs = loader.load()


gemma2 = ChatGroq(
    temperature=0.2,
    model="gemma2-9b-it",
)



summarize_prompt = ChatPromptTemplate.from_messages([
    ('system', '''Summarize the following paper in Korean.
Emphasize the uniqueness and contribution of the paper.
Answer should be in 20 sentences.
Please Answer in Korean.
'''),
    ('user', '''{text}''')])

print(len(docs[0].page_content))
summarize_prompt.format_messages(text=docs[0].page_content)

16713


[SystemMessage(content='Summarize the following paper in Korean.\nEmphasize the uniqueness and contribution of the paper.\nAnswer should be in 20 sentences.\nPlease Answer in Korean.\n', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='\n\nLlama 3.2: Revolutionizing edge AI and vision with open, customizable models\n\n\n\n\n\n\n\n\n\n\nOur approachResearchProduct experiencesLlamaBlogTry Meta AIFEATUREDLarge Language Model Llama 3.2: Revolutionizing edge AI and vision with open, customizable modelsSeptember 25, 2024•15 minute readTakeaways:Today, we’re releasing Llama 3.2, which includes small and medium-sized vision LLMs (11B and 90B), and lightweight, text-only models (1B and 3B) that fit onto edge and mobile devices, including pre-trained and instruction-tuned versions.The Llama 3.2 1B and 3B models support context length of 128K tokens and are state-of-the-art in their class for on-device use cases like summarization, instruction following, and rewriting tasks run

In [8]:
summarize_chain = summarize_prompt | gemma2 | StrOutputParser()
summary = summarize_chain.invoke(docs[0].page_content)
print(summary)

## Llama 3.2: 핵심 내용 요약 및 차별성 강조 (20문장)

Meta는 오픈소스, 맞춤형 모델인 Llama 3.2를 출시하여 에지 AI 및 비전 분야에 혁신을 가져올 것이라고 발표했습니다. Llama 3.2는 11B 및 90B 크기의 비전 LLM과 1B 및 3B 크기의 가벼운 텍스트 전용 모델을 포함하며, Qualcomm 및 MediaTek 하드웨어와 Arm 프로세서를 위한 최적화된 버전을 제공합니다. 

이 모델은 텍스트 및 이미지를 이해하고 처리할 수 있는 능력을 갖추고 있으며, 이는 문서 이해, 이미지 설명, 이미지 기반 질문 답변 등 다양한 비전 작업에 적용될 수 있습니다. 특히, Llama 3.2는 텍스트 모델과 비슷한 성능을 유지하면서도 이미지 인코더를 통합하여 새로운 비전 작업을 수행할 수 있도록 설계되었습니다.

Llama 3.2의 주요 특징은 다음과 같습니다:

* **오픈소스 및 맞춤형:** 개발자들이 모델을 자유롭게 사용하고 수정할 수 있도록 지원합니다.
* **에지 및 모바일 기기 지원:** 1B 및 3B 모델은 에지 및 모바일 기기에 적합한 크기로 설계되었습니다.
* **강력한 비전 이해 능력:** 11B 및 90B 모델은 이미지 인식, 이해, 그리고 설명을 위한 강력한 능력을 제공합니다.
* **다양한 환경 지원:** Llama Stack를 통해 다양한 환경 (온프레미스, 클라우드, 단일 노드, 에지)에서 모델을 사용할 수 있습니다.
* **안전성 강화:** Llama Guard 3를 통해 안전하고 책임감 있는 AI 개발을 지원합니다.

Llama 3.2는 기존 Llama 모델보다 더욱 강력하고 다용도적인 모델이며, 오픈소스 접근 방식을 통해 AI 기술 접근성을 높이고 혁신을 촉진할 것으로 기대됩니다.

특히, Llama 3.2는 다음과 같은 측면에서 차별성을 가지고 있습니다:

* **비전 모델 지원:** Llama 3.2는 이미지 처리 능력을 갖춘 최초의 Llama 모델로, 비전 분야에서 새로운 가능성을 열었습니다.
* 

해당 방법은 매우 간단하지만, Context 길이를 넘어서는 경우 에러가 발생합니다.

## Map-Reduce
LangChain의 PyMuPdfLoader를 이용하여 임의의 PDF를 요약해 보겠습니다.

In [ ]:
# # Password 있는 PDF 열기
# from langchain.document_loaders import PyPDFLoader
# path_material = './암호화된파일.pdf'

# pypdf_loader = PyPDFLoader(path_material, password='비밀번호')
# material_pages = pypdf_loader.load()

In [9]:
from langchain_community.document_loaders import PyMuPDFLoader
import urllib.request

paper_URL = "https://arxiv.org/pdf/2410.05983"


# 외부 링크에서 PDF 파일을 다운로드하는 코드
urllib.request.urlretrieve(
    paper_URL,
    filename="paper.pdf"
)
path = './paper.pdf'

In [10]:
from langchain_core.documents import Document


loader = PyMuPDFLoader(path)
# 페이지별로 저장
pages = loader.load()

# 코퍼스에 모두 결합
corpus = Document(page_content='')
for page in pages:
    corpus.page_content += page.page_content
len(corpus.page_content)

97365

긴 Context를 처리하기 위해, 전체 코퍼스를 작은 단위로 쪼개 보겠습니다.

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
document_list = text_splitter.split_documents([corpus])
len(document_list)

11

이후 Map-Reduce와 Refine을 사용할 수 있습니다.

## Map-Reduce

In [12]:
from tqdm import tqdm

# Map 과정 : 각 문서에 대해 요약을 생성합니다.

map_prompt = ChatPromptTemplate.from_messages([
    ('system', '''주어진 논문의 내용을 읽고 한국어로 요약하세요.
요약은 1개의 문단과 문단별 6개의 문장으로 작성하세요.
답변은 한국어로 작성하세요.
같은 내용을 반복하지 마세요.
'''),
    ('user', '''{text}''')])

map_chain  = map_prompt | gemma2 | StrOutputParser()

raw_summaries = []
for i in tqdm(range(len(document_list))):
    response = map_chain.invoke(document_list[i].page_content)
    raw_summaries.append(response)
    print('')
    print('#',i)
    print(response)
    print('===========================')


  9%|▉         | 1/11 [00:01<00:13,  1.38s/it]


# 0
본 논문은 장문 맥락을 처리할 수 있는 대규모 언어 모델(LLM)을 기반으로 한 정보 검색 강화 생성(RAG) 시스템의 새로운 과제와 해결책을 제시합니다. 

장문 맥락 LLM을 RAG에 적용할 때, 단순히 검색된 문서의 개수를 늘리는 것이 성능 향상을 보장하지 않는다는 것을 발견했습니다. 오히려 검색된 문서의 개수가 증가함에 따라 성능이 감소하는 경우도 있었습니다. 이는 검색된 문서 중에서도 관련성이 낮거나 오히려 오류를 유발할 수 있는 "hard negatives"가 LLM의 생성 과정에 부정적인 영향을 미치기 때문입니다. 

논문에서는 이러한 문제를 해결하기 위해 "retrieval reordering", "implicit robustness fine-tuning", "explicit relevance fine-tuning" 세 가지 방법을 제안합니다. 첫 번째 방법은 검색된 문서의 순서를 재정렬하여 LLM이 더욱 관련성 있는 정보에 집중하도록 유도합니다. 두 번째 방법은 LLM을 노이즈가 포함된 데이터로 fine-tuning하여 hard negatives에 대한 내성을 향상시킵니다. 마지막 방법은 LLM이 검색된 문서를 분석하고 관련 정보를 명시적으로 식별하도록 학습시켜 LLM의 정보 필터링 능력을 강화합니다. 

이러한 방법들은 장문 맥락 LLM을 기반으로 한 RAG 시스템의 성능을 향상시키는 데 효과적임을 보여줍니다. 또한, 데이터 분포, 사용된 검색 알고리즘, 학습 맥락 길이와 같은 RAG-specific LLM fine-tuning에 영향을 미치는 요소들을 분석하여 실질적인 지침을 제공합니다.





 18%|█▊        | 2/11 [00:02<00:09,  1.08s/it]


# 1
본 논문은 장문맥 LLM(Long-Context LLMs)을 기반으로 한 RAG(Retrieval-Augmented Generation) 시스템에서 발생하는 과제들을 분석하고 해결 방안을 제시합니다. 

첫째, 장문맥 LLM이 처리할 수 있는 답변 텍스트의 길이가 증가함에 따라 RAG 성능이 증가하지만, 특정 지점 이후에는 감소하는 경향을 보인다는 것을 발견했습니다. 이는 강력한 검색기(retriever)가 더 많은 관련 텍스트를 검색하지만, 이 중 일부는 실제로는 관련성이 낮거나 오히려 오답을 유발하는 "hard negatives"를 포함할 수 있기 때문입니다. 둘째, 높은 정확도를 보이는 검색기가 "hard negatives"를 더 많이 검색하여 LLM 성능에 더 큰 부정적인 영향을 미칠 수 있다는 것을 알 수 있습니다. 즉, 단순히 검색 결과의 양보다 검색 결과의 질, 특히 "hard negatives"의 양과 특성이 LLM 성능에 큰 영향을 미친다는 것을 시사합니다. 

이러한 연구 결과를 바탕으로, 장문맥 LLM을 RAG에 적용할 때 "hard negatives"를 효과적으로 처리하고 LLM이 검색된 정보를 효율적으로 활용할 수 있도록 하는 새로운 방법론 개발이 필요함을 제시합니다.





 27%|██▋       | 3/11 [00:03<00:09,  1.13s/it]

...


# 8
본 논문은 Long-Context LLMs를 활용한 RAG(Retrieval Augmented Generation)의 새로운 접근 방식을 제시하며, 특히 중간 단계의 추론을 통한 RAG 성능 향상에 초점을 맞추고 있습니다. 

첫째, 논문은 다양한 데이터셋(NQ, Wizard of Wikipedia, FEVER, MMLU)에 대한 RAG 학습을 위한 새로운 지시문 템플릿을 제안합니다. 둘째, Gemini-1.5-pro 모델을 사용하여 중간 단계 추론을 생성하도록 지시하는 프롬프트를 설계하고, 이를 통해 RAG 모델의 추론 능력을 향상시키는 방법을 제시합니다. 셋째, 다양한 지시문 템플릿과 프롬프트를 사용하여 실험을 진행하고, 결과를 분석하여 중간 단계 추론이 RAG 성능에 미치는 영향을 규명합니다. 넷째, 데이터 증강 기법을 적용하여 RAG 모델의 성능을 더욱 향상시키는 사례 연구를 제시합니다. 다섯째, 실험 결과를 통해 중간 단계 추론이 RAG 모델의 정확도와 품질을 향상시키는 효과를 입증합니다. 마지막으로, 향후 연구 방향으로 중간 단계 추론을 더욱 발전시키고 다양한 RAG 도메인에 적용할 수 있는 가능성을 제시합니다.





 91%|█████████ | 10/11 [01:07<00:10, 10.25s/it]


# 9
본 논문은 다양한 규모의 LLMs(Large Language Models)를 사용하여 RAG(Retrieval Augmented Generation) 기법을 적용하여 질문 답변 작업 성능을 향상시키는 방법을 연구합니다. 특히, Gemma-2-9B, Mistral-Nemo-12B와 같은 모델들을 사용하여 데이터 증강된 RAG fine-tuning 기법을 적용하고, 이를 통해 기존의 chat LLM fine-tuning이나 직접 질문-답변 쌍으로 fine-tuning한 방법보다 성능이 향상됨을 보여줍니다. 

첫째, Gemma-2-9B 모델을 사용하여 다양한 데이터셋에서 RAG fine-tuning의 효과를 분석합니다. RAG fine-tuning은 기존의 chat LLM보다 질문 답변 정확도를 높이는 데 효과적이며, 특히 intermediate reasoning을 추가한 RAG FT w. Int 기법은 더욱 향상된 성능을 보여줍니다. 둘째, Mistral-Nemo-12B 모델을 사용하여 데이터 증강된 RAG fine-tuning 결과를 보여주며, Gemma-2-9B 모델과 마찬가지로 RAG fine-tuning이 성능 향상에 효과적임을 확인합니다. 셋째, 논문은 RAG 기법이 긴 텍스트 입력을 처리하는 데 효과적인 방법임을 보여주며, 이는 LLMs의 긴 텍스트 처리 능력을 향상시키는 데 기여할 수 있습니다. 

결론적으로, 본 논문은 데이터 증강된 RAG fine-tuning 기법이 LLMs의 질문 답변 성능을 향상시키는 효과적인 방법임을 보여주며, 특히 긴 텍스트 입력 처리에 유용함을 강조합니다.





100%|██████████| 11/11 [01:21<00:00,  7.45s/it]


# 10
본 논문은 긴 입력에 대한 RAG(Retrieval Augmented Generation)의 성능을 향상시키기 위해 Long-Context LLM(Large Language Model)을 사용하는 방법을 연구합니다. 

먼저, Mistral-Nemo-12B와 Gemini-1.0-Pro와 같은 다양한 LLM 기반 모델을 사용하여 RAG 특화 튜닝 방법을 평가합니다.  결과는 RAG 특화 튜닝이 LLM의 RAG 성능을 향상시키는 데 효과적임을 보여줍니다. 특히, "intermediate reasoning step"을 포함하는 RAG FT w. Int 방법이 가장 우수한 성능을 보입니다. 

또한, RAG 특화 튜닝 데이터의 크기가 LLM 성능에 미치는 영향을 분석합니다.  결과는 데이터 크기가 증가함에 따라 RAG 성능이 향상됨을 보여주며, 특히 200k개의 데이터를 사용했을 때 가장 높은 성능을 보입니다. 

마지막으로, RAG 특화 데이터와 일반적인 SFT(Supervised Fine-Tuning) 데이터를 결합하여 LLM을 훈련하는 방법을 제시하고, 이 방법이 LLM의 RAG 성능을 향상시키면서도 일반적인 언어 처리 능력을 유지할 수 있음을 보여줍니다. 





In [13]:
raw_summaries

['본 논문은 장문 맥락을 처리할 수 있는 대규모 언어 모델(LLM)을 기반으로 한 정보 검색 강화 생성(RAG) 시스템의 새로운 과제와 해결책을 제시합니다. \n\n장문 맥락 LLM을 RAG에 적용할 때, 단순히 검색된 문서의 개수를 늘리는 것이 성능 향상을 보장하지 않는다는 것을 발견했습니다. 오히려 검색된 문서의 개수가 증가함에 따라 성능이 감소하는 경우도 있었습니다. 이는 검색된 문서 중에서도 관련성이 낮거나 오히려 오류를 유발할 수 있는 "hard negatives"가 LLM의 생성 과정에 부정적인 영향을 미치기 때문입니다. \n\n논문에서는 이러한 문제를 해결하기 위해 "retrieval reordering", "implicit robustness fine-tuning", "explicit relevance fine-tuning" 세 가지 방법을 제안합니다. 첫 번째 방법은 검색된 문서의 순서를 재정렬하여 LLM이 더욱 관련성 있는 정보에 집중하도록 유도합니다. 두 번째 방법은 LLM을 노이즈가 포함된 데이터로 fine-tuning하여 hard negatives에 대한 내성을 향상시킵니다. 마지막 방법은 LLM이 검색된 문서를 분석하고 관련 정보를 명시적으로 식별하도록 학습시켜 LLM의 정보 필터링 능력을 강화합니다. \n\n이러한 방법들은 장문 맥락 LLM을 기반으로 한 RAG 시스템의 성능을 향상시키는 데 효과적임을 보여줍니다. 또한, 데이터 분포, 사용된 검색 알고리즘, 학습 맥락 길이와 같은 RAG-specific LLM fine-tuning에 영향을 미치는 요소들을 분석하여 실질적인 지침을 제공합니다.\n\n\n',
 '본 논문은 장문맥 LLM(Long-Context LLMs)을 기반으로 한 RAG(Retrieval-Augmented Generation) 시스템에서 발생하는 과제들을 분석하고 해결 방안을 제시합니다. \n\n첫째, 장문맥 LLM이 처리할 수 있는 답변 텍스트의 길이가 증가함에 따라 RAG 성능이 증가하지만, 특정 지점 이후에

In [14]:
# Reduce 과정 : 각 문서의 요약을 하나로 합칩니다.
reduce_prompt = ChatPromptTemplate.from_messages([

    ('system', '''
Generate a summary of the following text that includes the following elements:

* A title that accurately reflects the content of the text.
* An introduction paragraph that provides an overview of the topic.
* Bullet points that list the key points of the text.
* A conclusion paragraph that summarizes the main points of the text.

Answer in Korean.
'''),
    ('user', '''{text}
---
요약(In Korean):
''')])

reduce_chain = reduce_prompt | gemma2 | StrOutputParser()

summary = reduce_chain.invoke('\n---\n'.join(raw_summaries))

print(summary)


## 장문 맥락 LLM 기반 RAG 시스템 성능 향상 방안 연구: "hard negatives" 문제 해결과 효과적인 fine-tuning 방법 제시

본 논문은 장문 맥락을 이해하는 대규모 언어 모델(LLM)을 Retrieval-Augmented Generation(RAG) 시스템에 적용할 때 발생하는 새로운 과제들을 분석하고 해결책을 제시합니다. 

**핵심 내용:**

* **장문 맥락 LLM의 RAG 적용 시 "hard negatives" 문제:** 단순히 검색된 문서의 개수를 늘리는 것이 성능 향상을 보장하지 않는다는 것을 발견했습니다. 오히려 검색된 문서 중 관련성이 낮거나 오히려 오류를 유발하는 "hard negatives"가 LLM의 생성 과정에 부정적인 영향을 미치는 것을 확인했습니다.
* **"hard negatives" 문제 해결을 위한 세 가지 방법 제안:**
    * **retrieval reordering:** 검색된 문서의 순서를 재정렬하여 LLM이 더욱 관련성 있는 정보에 집중하도록 유도합니다.
    * **implicit robustness fine-tuning:** LLM을 노이즈가 포함된 데이터로 fine-tuning하여 hard negatives에 대한 내성을 향상시킵니다.
    * **explicit relevance fine-tuning:** LLM이 검색된 문서를 분석하고 관련 정보를 명시적으로 식별하도록 학습시켜 정보 필터링 능력을 강화합니다.
* **RAG-specific fine-tuning의 중요성:** 데이터 분포, 사용된 검색 알고리즘, 학습 맥락 길이와 같은 요소들이 RAG 시스템의 성능에 영향을 미친다는 것을 분석하고 실질적인 지침을 제공합니다.

**결론:**

본 논문은 장문 맥락 LLM을 기반으로 한 RAG 시스템의 성능을 향상시키는 데 효과적인 방법을 제시합니다. 특히, "hard negatives" 문제를 인식하고 이를 해결하기 위한 retrieval reordering, RAG-specif

## Refine

Refine은 청크를 순서대로 참고하며, 매 시점 요약문을 작성합니다.   
요약문과 새로운 청크를 비교하여, 요약문을 업데이트합니다.

In [15]:
first_summarize_prompt = ChatPromptTemplate.from_messages([
    ('system', '''당신은 인공지능 전문가입니다.
주어진 논문의 내용을 읽고 한국어로 요약하세요.
요약은 1개의 문단과 문단별 5개의 문장으로 작성하세요.
답변은 한국어로 작성하세요.'''),
    ('user', '''{text}''')])



X = first_summarize_prompt.format_messages(text=document_list[0])

intermediate_summary = gemma2.invoke(X).content
print(intermediate_summary)

본 논문은 장문 맥락을 가진 대규모 언어 모델(LLM)을 사용하는 Retrieval-Augmented Generation(RAG) 시스템의 새로운 과제와 해결책을 제시합니다.  장문 맥락 LLM은 더 많은 정보를 활용할 수 있지만, 이는 오히려 성능 저하로 이어질 수 있습니다. 이는 LLM이 많은 정보 중에서도 관련성이 낮은 정보(hard negatives)에 영향을 받기 때문입니다.  

논문은 장문 맥락 LLM을 사용하는 RAG 시스템에서 hard negatives가 성능에 미치는 부정적인 영향을 분석합니다. 단순히 더 많은 정보를 제공하는 것이 항상 좋은 결과를 가져오지 않는다는 것을 보여주며, 강력한 검색기(retriever)를 사용하는 것조차 문제를 악화시킬 수 있다는 것을 발견했습니다. 이러한 문제를 해결하기 위해 논문은 다음과 같은 세 가지 방법을 제안합니다. 첫째, 검색된 문서의 순서를 재정렬하여 LLM의 주의를 더 관련성 있는 정보로 유도합니다. 둘째, LLM을 잠재적인 노이즈를 포함한 쿼리와 검색된 문서 데이터로 fine-tuning하여 hard negatives에 대한 내성을 향상시킵니다. 셋째, LLM을 중간 단계의 추론을 통해 검색된 문서를 분석하고 관련 정보를 명시적으로 식별하도록 학습하여 LLM의 관련성 파악 능력을 향상시킵니다. 

이러한 방법들은 데이터 분포, 사용된 검색기, 학습 맥락 길이와 같은 다양한 요소에 대한 광범위한 연구를 통해 개발되었습니다. 논문은 장문 맥락 LLM을 사용하는 RAG 시스템의 성능을 향상시키는 데 기여하며, 이러한 시스템의 잠재력을 더욱 탐구할 수 있는 방향을 제시합니다. 





In [17]:
# Refine Prompt

refine_prompt = ChatPromptTemplate.from_messages([
    ('system', '''당신은 인공지능 전문가입니다.
논문의 현재 시점까지의 한국어 요약이 주어집니다.
이를 읽고, 새롭게 주어지는 내용과 비교하여 한국어 요약을 보완하거나 수정하세요.
전체 요약은 10문장 이내로 작성하세요.
'''),
    ('user', '''현재 시점까지의 요약: {previous_summary}
---
새로운 내용: {new_text}''')])


refine_chain = refine_prompt | gemma2 | StrOutputParser()

for i in tqdm(range(1, len(document_list))):
    intermediate_summary = refine_chain.invoke(
        {'previous_summary':intermediate_summary,
         'new_text':document_list[i].page_content})
    print('')
    print(intermediate_summary)
    print('=======================')
# 길이를 지정하지 않으면 오래 걸릴 수 있습니다.



 10%|█         | 1/10 [00:02<00:20,  2.29s/it]


본 논문은 장문 맥락을 이해하는 대규모 언어 모델(LLM)을 사용하는 Retrieval-Augmented Generation(RAG) 시스템의 새로운 과제와 해결책을 제시합니다. 장문 맥락 LLM은 더 많은 정보를 활용할 수 있지만, 이는 오히려 성능 저하로 이어질 수 있습니다. 이는 LLM이 많은 정보 중에서도 관련성이 낮은 정보(hard negatives)에 영향을 받기 때문입니다. 

논문은 장문 맥락 LLM을 사용하는 RAG 시스템에서 hard negatives가 성능에 미치는 부정적인 영향을 분석합니다. 단순히 더 많은 정보를 제공하는 것이 항상 좋은 결과를 가져오지 않는다는 것을 보여주며, 강력한 검색기(retriever)를 사용하는 것조차 문제를 악화시킬 수 있다는 것을 발견했습니다. 이러한 문제를 해결하기 위해 논문은 다음과 같은 세 가지 방법을 제안합니다. 첫째, 검색된 문서의 순서를 재정렬하여 LLM의 주의를 더 관련성 있는 정보로 유도합니다. 둘째, LLM을 잠재적인 노이즈를 포함한 쿼리와 검색된 문서 데이터로 fine-tuning하여 hard negatives에 대한 내성을 향상시킵니다. 셋째, LLM을 중간 단계의 추론을 통해 검색된 문서를 분석하고 관련 정보를 명시적으로 식별하도록 학습하여 LLM의 관련성 파악 능력을 향상시킵니다. 

이러한 방법들은 데이터 분포, 사용된 검색기, 학습 맥락 길이와 같은 다양한 요소에 대한 광범위한 연구를 통해 개발되었습니다. 논문은 장문 맥락 LLM을 사용하는 RAG 시스템의 성능을 향상시키는 데 기여하며, 이러한 시스템의 잠재력을 더욱 탐구할 수 있는 방향을 제시합니다. 특히, 강력한 검색기가 더 많은 관련 정보를 제공하지만, 이로 인해 발생하는 hard negatives 문제를 해결하기 위한 방법을 제시합니다. 





 20%|██        | 2/10 [00:03<00:13,  1.69s/it]


본 논문은 장문 맥락을 이해하는 대규모 언어 모델(LLM)을 사용하는 Retrieval-Augmented Generation(RAG) 시스템에서 발생하는 새로운 과제와 해결책을 제시합니다. 장문 맥락 LLM은 더 많은 정보를 활용할 수 있지만, 이는 오히려 성능 저하로 이어질 수 있습니다. 이는 LLM이 많은 정보 중에서도 관련성이 낮은 정보(hard negatives)에 영향을 받기 때문입니다. 

논문은 장문 맥락 LLM을 사용하는 RAG 시스템에서 hard negatives가 성능에 미치는 부정적인 영향을 분석합니다. 단순히 더 많은 정보를 제공하는 것이 항상 좋은 결과를 가져오지 않는다는 것을 보여주며, 강력한 검색기(retriever)를 사용하는 것조차 문제를 악화시킬 수 있다는 것을 발견했습니다. 이러한 문제를 해결하기 위해 논문은 다음과 같은 세 가지 방법을 제안합니다. 첫째, 검색된 문서의 순서를 재정렬하여 LLM의 주의를 더 관련성 있는 정보로 유도합니다. 둘째, LLM을 잠재적인 노이즈를 포함한 쿼리와 검색된 문서 데이터로 fine-tuning하여 hard negatives에 대한 내성을 향상시킵니다. 셋째, LLM을 중간 단계의 추론을 통해 검색된 문서를 분석하고 관련 정보를 명시적으로 식별하도록 학습하여 LLM의 관련성 파악 능력을 향상시킵니다. 

특히, 강력한 검색기가 더 많은 관련 정보를 제공하지만, 이로 인해 발생하는 hard negatives 문제를 해결하기 위한 방법을 제시합니다.  논문은 검색된 문서의 순서를 재정렬하는 방법을 통해 LLM이 hard negatives에 영향을 받지 않도록 유도하고, LLM을 쿼리와 검색된 문서 데이터로 fine-tuning하여 hard negatives에 대한 내성을 높이는 방법을 제안합니다. 또한, LLM이 검색된 문서를 분석하고 관련 정보를 명시적으로 식별하도록 학습하는 방법을 통해 LLM의 관련성 파악 능력을 향상시키는 방법을 제시합니다. 





 30%|███       | 3/10 [00:05<00:11,  1.65s/it]

...


본 논문은 장문 맥락을 이해하는 대규모 언어 모델(LLM)을 사용하는 Retrieval-Augmented Generation(RAG) 시스템에서 발생하는 hard negatives 문제를 해결하기 위한 방법을 제시합니다. 

hard negatives는 LLM이 관련성이 낮은 정보에 영향을 받아 성능 저하를 초래할 수 있습니다. 논문은 강력한 검색기(retriever)를 사용하더라도 hard negatives 문제가 발생하며, 이는 LLM의 성능을 저하시킬 수 있다는 것을 보여줍니다.  

논문은 이러한 문제를 해결하기 위해 세 가지 방법을 제안합니다. 첫째, 검색된 문서의 순서를 재정렬하여 LLM의 주의를 더 관련성 있는 정보로 유도합니다. 둘째, LLM을 잠재적인 노이즈를 포함한 쿼리와 검색된 문서 데이터로 fine-tuning하여 hard negatives에 대한 내성을 향상시킵니다. 셋째, LLM을 중간 단계의 추론을 통해 검색된 문서를 분석하고 관련 정보를 명시적으로 식별하도록 학습하여 LLM의 관련성 파악 능력을 향상시킵니다. 

특히, weaker retrievers 또는 무작위 샘플링과 같은 방법을 통해 hard negatives를 줄일 수 있다는 것을 보여줍니다. 또한, hard negatives가 LLM의 성능에 부정적인 영향을 미치는 주요 요인이며, 이를 해결하기 위한 노력이 필요함을 강조합니다. 

본 논문은 Implicit RAG Fine-Tuning과 RAG Finetuning with Intermediate Reasoning 두 가지 새로운 방법을 제시합니다. Implicit RAG Fine-Tuning은 LLM을 쿼리와 검색된 문서를 함께 입력으로 사용하여 fine-tuning하는 방법이며, RAG Finetuning with Intermediate Reasoning은 LLM이 검색된 문서를 분석하고 추론을 통해 관련 정보를 명시적으로 식별하도록 학습하는 방법입니다. 





 90%|█████████ | 9/10 [01:21<00:12, 12.69s/it]


본 논문은 장문 맥락을 이해하는 대규모 언어 모델(LLM)을 사용하는 Retrieval-Augmented Generation(RAG) 시스템에서 발생하는 hard negatives 문제를 해결하기 위한 방법을 제시합니다. 

hard negatives는 LLM이 관련성이 낮은 정보에 영향을 받아 성능 저하를 초래할 수 있습니다. 논문은 강력한 검색기(retriever)를 사용하더라도 hard negatives 문제가 발생하며, 이는 LLM의 성능을 저하시킬 수 있다는 것을 보여줍니다. 

논문은 이러한 문제를 해결하기 위해 세 가지 방법을 제안합니다. 첫째, 검색된 문서의 순서를 재정렬하여 LLM의 주의를 더 관련성 있는 정보로 유도합니다. 둘째, LLM을 잠재적인 노이즈를 포함한 쿼리와 검색된 문서 데이터로 fine-tuning하여 hard negatives에 대한 내성을 향상시킵니다. 셋째, LLM을 중간 단계의 추론을 통해 검색된 문서를 분석하고 관련 정보를 명시적으로 식별하도록 학습하여 LLM의 관련성 파악 능력을 향상시킵니다. 

특히, weaker retrievers 또는 무작위 샘플링과 같은 방법을 통해 hard negatives를 줄일 수 있다는 것을 보여줍니다. 또한, hard negatives가 LLM의 성능에 부정적인 영향을 미치는 주요 요인이며, 이를 해결하기 위한 노력이 필요함을 강조합니다. 

본 논문은 Implicit RAG Fine-Tuning과 RAG Finetuning with Intermediate Reasoning 두 가지 새로운 방법을 제시합니다. Implicit RAG Fine-Tuning은 LLM을 쿼리와 검색된 문서를 함께 입력으로 사용하여 fine-tuning하는 방법이며, RAG Finetuning with Intermediate Reasoning은 LLM이 검색된 문서를 분석하고 추론을 통해 관련 정보를 명시적으로 식별하도록 학습하는 방법입니다. 

**추가적으로, 논문은 Gemma-2-9B와 Mistral-N

100%|██████████| 10/10 [01:38<00:00,  9.88s/it]


본 논문은 장문 맥락 이해 능력을 갖춘 대규모 언어 모델(LLM)을 사용하는 Retrieval-Augmented Generation(RAG) 시스템에서 발생하는 hard negatives 문제를 해결하기 위한 방법을 제시합니다. 

hard negatives는 LLM이 관련성이 낮은 정보에 영향을 받아 성능 저하를 초래할 수 있습니다. 논문은 강력한 검색기(retriever)를 사용하더라도 hard negatives 문제가 발생하며, 이는 LLM의 성능을 저하시킬 수 있다는 것을 보여줍니다. 

논문은 이러한 문제를 해결하기 위해 세 가지 방법을 제안합니다. 첫째, 검색된 문서의 순서를 재정렬하여 LLM의 주의를 더 관련성 있는 정보로 유도합니다. 둘째, LLM을 잠재적인 노이즈를 포함한 쿼리와 검색된 문서 데이터로 fine-tuning하여 hard negatives에 대한 내성을 향상시킵니다. 셋째, LLM을 중간 단계의 추론을 통해 검색된 문서를 분석하고 관련 정보를 명시적으로 식별하도록 학습하여 LLM의 관련성 파악 능력을 향상시킵니다. 

특히, weaker retrievers 또는 무작위 샘플링과 같은 방법을 통해 hard negatives를 줄일 수 있다는 것을 보여줍니다. 또한, hard negatives가 LLM의 성능에 부정적인 영향을 미치는 주요 요인이며, 이를 해결하기 위한 노력이 필요함을 강조합니다. 

본 논문은 Implicit RAG Fine-Tuning과 RAG Finetuning with Intermediate Reasoning 두 가지 새로운 방법을 제시합니다. Implicit RAG Fine-Tuning은 LLM을 쿼리와 검색된 문서를 함께 입력으로 사용하여 fine-tuning하는 방법이며, RAG Finetuning with Intermediate Reasoning은 LLM이 검색된 문서를 분석하고 추론을 통해 관련 정보를 명시적으로 식별하도록 학습하는 방법입니다. 

**추가적으로, 논문은 Gemma-2-9B와 Mistr